Model to detect PUSHUP with MediaPipe and OpenCV.

Upload the video input in:

```
video = " "
```
Run all cells and the output will be saved in the format `output.avi` .


Install and import required libraries

In [ ]:
!pip install OpenCV-Python
!pip install MediaPipe
!pip install imutils

In [ ]:
import cv2 as cv
import mediapipe as mp
import imutils
import math

Upload the video here.

In [ ]:
video = " "

Declare required variables

In [ ]:
# identify the landmarks
mp_pose = mp.solutions.pose

# position of elbows and shoulders
position = None

# video input
cap = cv.VideoCapture(video)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# saving the output
result_video = "output.avi"
out = cv.VideoWriter(result_video, cv.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (frame_width, frame_height))

# params to put text in frame
font = cv.FONT_HERSHEY_SIMPLEX
fontScale = 1
thickness = int(2)

Call MediaPipe pose estimation model and preprocess the input.

In [ ]:
with mp_pose.Pose(
    min_detection_confidence = 0.7,
    min_tracking_confidence = 0.7) as pose:

    while cap.isOpened():
      ret, frame = cap.read()

      if not ret:
        print("can't access frame")
        break
      image = imutils.resize(frame, width=500)
      image = cv.cvtColor(cv.flip(image, 1), cv.COLOR_BGR2RGB)
      result = pose.process(image)

      landmarks = []

      if result.pose_landmarks:
        for id, im in enumerate(result.pose_landmarks.landmark):
          h, w, _ = image.shape

          # finding exact corrdinates of body points.
          X, Y = int(im.x * w), int(im.y * h)
          landmarks.append([id, X, Y])

      if len(landmarks) != 0:
          if (landmarks[12][2] and landmarks[11][2] >= landmarks[14][2] and landmarks[13][2]):
            position = "down"

          if (landmarks[12][2] and landmarks[13][2] <= landmarks[14][2] and landmarks[13][2]) and position == "down":
            position = "up"

      # Displaying the output
      if position == "down" or position == "up":
          cv.putText(frame, "PUSH UP", (50, 50), font, fontScale, (0, 255, 0), 4)
      else:
          cv.putText(frame, "NOT A PUSHUP", (50, 50), font, fontScale, (0, 255, 0), 4)

      form = cv.putText(frame, f"POSITION OF YOUR BODY: {position}", (500, 40), font, fontScale, (0, 0, 255), thickness)

      # write the output
      out.write(frame)

      if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()